In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook
from pyecharts.globals import ThemeType
csvPath='csv/'

provinceDataAll=pd.read_csv(csvPath+'Province_data.csv')
provinceDataAll.index=provinceDataAll['update_date']
provinceDataAll=provinceDataAll.drop('update_date',axis=1)
provinceDataAll=provinceDataAll.fillna(0)
provinceDataAll2=pd.read_csv(csvPath+'各省重症死亡治愈累计数值.csv')
provinceDataAll2.index=provinceDataAll2['指标名称']
provinceDataAll2=provinceDataAll2.drop('指标名称',axis=1)
provinceDataAll2=provinceDataAll2.fillna(0)
# provinceDataAll2

In [2]:
provinceCured=provinceDataAll2.T[1::3]
# provinceCured=provinceCured.reindex(provinceCured['2020/5/23'].sort_values(ascending=False).index)
provinceCured=provinceCured.T
provinceCured.columns= [i.split(':', 1)[0] for i in provinceCured.columns]
# provinceCured

In [3]:
provinceDead=provinceDataAll2.T[2::3]
provinceDead=provinceDead.T
provinceDead.columns= [i.split(':', 1)[0] for i in provinceDead.columns]
pd.set_option('display.max_columns' , None)
# provinceDead

In [4]:
#计算全国数据
provinceDataAll['全国'] = provinceDataAll.apply(lambda x: x.sum(), axis=1)
provinceCured['全国'] = provinceCured.apply(lambda x: x.sum(), axis=1)
provinceDead['全国'] = provinceDead.apply(lambda x: x.sum(), axis=1)
# provinceDataAll.loc['sum'] = provinceDataAll.apply(lambda x: x.sum())
# provinceCured
# provinceDead
# provinceDataAll

In [5]:
# for i in provinceDataAll.columns:
#     if i not in provinceCured.columns:
#         provinceCured[i]=0
#     if i not in provinceDead.columns:
#         provinceDead[i]=0
# provinceDataAll.columns

In [6]:
#省份排序
provinceData_sorted=provinceDataAll.T
provinceData_sorted=provinceData_sorted.reindex(provinceData_sorted['2020/5/23'].sort_values(ascending=False).index)
provinceData_sorted=provinceData_sorted.T
# provinceData_sorted
# provinceDead

In [7]:
#计算增量
provinceData_sorted_increase=provinceData_sorted

provinceData_sorted_Shift = provinceData_sorted_increase.shift(1)
provinceData_sorted_Shift =provinceData_sorted_Shift.fillna(0)

provinceData_sorted_increase = provinceData_sorted_increase-provinceData_sorted_Shift
provinceData_sorted_increase.loc['sum'] = provinceData_sorted.loc['2020/5/23']
# provinceData_sorted_increase

In [8]:
import win32api,win32con
screenWidth=win32api.GetSystemMetrics(win32con.SM_CXSCREEN)   #获得屏幕分辨率X轴
screenHeight=win32api.GetSystemMetrics(win32con.SM_CYSCREEN)   #获得屏幕分辨率Y轴
# calendarData.columns
# calendarData.index.tolist()[:-2]

In [9]:
import datetime
import pyecharts.options as opts
from pyecharts.charts import Calendar, Tab, Line, Grid, Liquid, Bar, Page

calendarData = provinceData_sorted_increase
sum = calendarData.loc['sum']
sumAll = sum['全国']

tab = Tab()
tab2 = Tab()

for pr in calendarData.columns:
    
    begin = datetime.date(2020, 1, 16)
    end = datetime.date(2020, 5, 22)  
    
    l=provinceData_sorted_increase[pr].tolist()
      
    line = (
        Line(init_opts=opts.InitOpts(width=str(screenWidth-30)+"px", height=str(screenHeight*(2/3))+"px"))
        .add_xaxis(calendarData.index.tolist()[:-2])
        .add_yaxis("每日新增数据", l, is_smooth=True)
        .add_yaxis("累计确诊数据", provinceData_sorted[pr], is_smooth=True)
        .add_yaxis("累计死亡数据", provinceDead[pr], is_smooth=True)
        .add_yaxis("累计治愈数据", provinceCured[pr], is_smooth=True)
        .set_global_opts(
            title_opts=opts.TitleOpts(title=pr, pos_left="5%"),
            legend_opts=opts.LegendOpts(pos_left="center"),
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    )
    
#     bar = (
#         Bar()
#         .add_xaxis(calendarData.index.tolist())
#         .add_yaxis("每日新增数据", l)
#         .set_global_opts(
#             title_opts=opts.TitleOpts(title="日变化"),
#             datazoom_opts=[opts.DataZoomOpts()],
#         )
#     )
    
    
    data = [
        [str(begin + datetime.timedelta(days=i)), l[i]]
        for i in range((end - begin).days + 1)
    ]
    
    l.sort()
    if l[-2]-l[-3]>100:
        lMax=l[-3]
    else:
        lMax=l[-2]
    
    calendar=(
        Calendar(init_opts=opts.InitOpts())#width=str(screenWidth/2)+"px", height=str(screenHeight/2)+"px"
        .add(
            series_name="",
            yaxis_data=data,
            calendar_opts=opts.CalendarOpts(
                pos_top="80",
                pos_left="30",
                pos_right=str(screenWidth/2),
                range_=[begin,end],
#                 yearlabel_opts=opts.CalendarYearLabelOpts(is_show=False),
            ),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(pos_top="30", pos_left="center", title=pr),
            visualmap_opts=opts.VisualMapOpts(
                max_=lMax, min_=0, orient="horizontal", is_piecewise=False
            ),
        )
    )

    lq = (
        Liquid()
        .add("占全国比", [sum[pr]/sumAll*100, sum[pr]/sum['全国']*100], center=["80%", "50%"])
        .set_global_opts(title_opts=opts.TitleOpts(pos_bottom="30", pos_left="70%", title="            占全国比例*100\n\n（"+pr+"累计"+str(int(sum[pr]))+"/全国累计"+str(int(sumAll))+"*100）"))
    )
    
    
#     page=Page()
#     page.add(calendar,lq)
    
    
    grid = Grid(init_opts=opts.InitOpts(width=str(screenWidth-50)+"px", height=str(screenHeight*(3/5))+"px"))
#     grid.add(bar, grid_opts=opts.GridOpts(pos_left="0%"))
    grid.add(calendar, grid_opts=opts.GridOpts(pos_left="0%"))
    grid.add(lq, grid_opts=opts.GridOpts(pos_left="55%"))

    
#     grid2 = Grid(init_opts=opts.InitOpts(width=str(screenWidth-50)+"px", height=str(screenHeight)+"px"))
#     grid2.add(line, grid_opts=opts.GridOpts(pos_left="0%", pos_bottom="50%"))
#     grid2.add(calendar, grid_opts=opts.GridOpts(pos_left="0%", pos_top="50%"))
    
    tab.add(grid, pr)
    tab2.add(line, pr)
tab.render("各省日历图.html")
tab2.render("各省曲线图.html")

'C:\\Users\\Administrator\\各省曲线图.html'

In [15]:
pr="湖北"
line1 = (
    Line(init_opts=opts.InitOpts(width=str(screenWidth-30)+"px", height=str(screenHeight*(2/3))+"px"))
    .add_xaxis(calendarData.index.tolist()[:-1])
    .add_yaxis("每日新增数据", l, is_smooth=True)
    .add_yaxis("累计确诊数据", provinceData_sorted[pr], is_smooth=True)
    .add_yaxis("累计死亡数据", provinceDead[pr], is_smooth=True)
    .add_yaxis("累计治愈数据", provinceCured[pr], is_smooth=True)
    .set_global_opts(
        title_opts=opts.TitleOpts(title=pr, pos_left="5%"),
        legend_opts=opts.LegendOpts(pos_left="center"),
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)
line.render_notebook()